In [ ]:
!pip install -e .. > /dev/null 2>&1

In [ ]:
import os
import json
import gdown
import shutil
import astrohack

from astrohack.gdown_utils import gdown_data
from astrohack.gdown_utils import build_folder_structure

from astrohack.dio import open_pointing

In [ ]:
# Build folder structure & download 

datafolder = 'data'
resultsfolder = 'results'

build_folder_structure(datafolder, resultsfolder)

astrohack.gdown_utils.download('J1924-2914.ms.calibrated.split.SPW3', folder=datafolder, unpack=True)
astrohack.gdown_utils.download(file='extract_holog_verification.json')
astrohack.gdown_utils.download(file='holog_numerical_verification.json')

In [ ]:
from astrohack.astrohack_client import astrohack_local_client

log_parms = {'log_level':'DEBUG'}

client = astrohack_local_client(cores=2, memory_limit='8GB', log_parms=log_parms)

In [ ]:
def test_holog_obs_dictionary(holog_obs_dict):
    import os
    import json
    import copy
    
    import numpy as np

    from astrohack._utils._tools import _jsonify

    ref_holog_obj = {}
    ref_holog_obj = copy.deepcopy(holog_obs_dict)

    _jsonify(ref_holog_obj)

    with open(".holog_obs_dict.json") as json_file:
        holog_obj = json.load(json_file)
                          
    assert holog_obj == ref_holog_obj, "Error: holog_obs_descrition dictionary has changes unexpectedly."
    
def test_holog_diagnostics(cell_size, grid_size, number_of_digits=7):
    import json
    
    with open(".holog_diagnostic.json") as json_file:
        json_data = json.load(json_file)
        
    json_data['cell_size'] = [round(x, number_of_digits) for x in json_data['cell_size']]
        
    assert (json_data['cell_size'] == cell_size).all(), "Unexpected change in cell_size occured."
    assert (json_data['grid_size'] == grid_size).all(), "Unexpected change in grid_size occured."

    
def test_center_pixel(file, antenna, ddi, reference_center_pixels, number_of_digits=7):
    from astrohack.dio import open_image
    
    mds = open_image(file)[antenna][ddi]
    
    aperture_shape = mds.APERTURE.values.shape[-2], mds.APERTURE.values.shape[-1]
    beam_shape = mds.BEAM.values.shape[-2], mds.BEAM.values.shape[-1]    
    
    aperture_center_pixels = np.squeeze(mds.APERTURE.values[..., aperture_shape[0]//2, aperture_shape[1]//2])
    beam_center_pixels = np.squeeze(mds.BEAM.values[..., beam_shape[0]//2, beam_shape[1]//2])
    
    aperture_ref = list(map(complex, reference_center_pixels['aperture']))
    beam_ref = list(map(complex, reference_center_pixels['beam']))
    
    for i in range(len(aperture_ref)):
        # Should probably write a custom round function here
        assert round(aperture_ref[i].real, number_of_digits) == round(aperture_center_pixels[i].real, number_of_digits), "There has been a shift in aperture center pixel value(s)"
        assert round(beam_ref[i].real, number_of_digits) == round(beam_center_pixels[i].real, number_of_digits), "There has been a shift in beam center pixel value(s)"
    
        assert round(aperture_ref[i].imag, number_of_digits) == round(aperture_center_pixels[i].imag, number_of_digits), "There has been a shift in aperture center pixel value(s)"
        assert round(beam_ref[i].imag, number_of_digits) == round(beam_center_pixels[i].imag, number_of_digits), "There has been a shift in beam center pixel value(s)"
    
def get_center_pixel(file, antenna, ddi):
    from astrohack.dio import open_image
    
    mds = open_image(file)[antenna][ddi]
    
    aperture_shape = mds.APERTURE.values.shape[-2], mds.APERTURE.values.shape[-1]
    beam_shape = mds.BEAM.values.shape[-2], mds.BEAM.values.shape[-1]    
    
    aperture_center_pixels = mds.APERTURE.values[..., aperture_shape[0]//2, aperture_shape[1]//2]
    beam_center_pixels = mds.BEAM.values[..., beam_shape[0]//2, beam_shape[1]//2]
    
    return np.squeeze(aperture_center_pixels), np.squeeze(beam_center_pixels)

In [ ]:
import numpy as np

from astrohack.extract_holog import extract_holog

with open("extract_holog_verification.json") as file:
    json_dict = json.load(file)
    
holog_obs_dict = json_dict["alma"]

alma_ms = "data/J1924-2914.ms.calibrated.split.SPW3"
alma_holog = 'results/alma.split.holog.zarr'

extract_holog(
    ms_name=alma_ms,
    holog_name=alma_holog,
    data_column='DATA',
    parallel=False,
    overwrite=True,
    reuse_point_zarr=False
)

test_holog_obs_dictionary(holog_obs_dict)

In [ ]:
import numpy as np
from astrohack import holog

with open("holog_numerical_verification.json") as file:
    reference_dict = json.load(file)

grid_interpolation_mode = 'linear' #'nearest' 'linear' 'cubic'
chan_average = True
scan_average = True

holog(
    holog_name=alma_holog, 
    padding_factor=50, 
    grid_interpolation_mode=grid_interpolation_mode,
    chan_average = chan_average,
    scan_average = scan_average,
    overwrite=True,
    phase_fit=True,
    apply_mask=True,
    to_stokes=True,
    parallel=True
)
    
test_center_pixel(file="results/alma.split.image.zarr", antenna="ant_DV13", ddi="ddi_0", reference_center_pixels=reference_dict["alma"]['pixels'])

test_holog_diagnostics(
    cell_size = np.array(reference_dict["alma"]['cell_size']),
    grid_size = np.array(reference_dict["alma"]['grid_size']),
    number_of_digits=6
)

In [ ]:
import xarray as xr
import matplotlib.pyplot as plt
import scipy
import matplotlib.patches as patches

from scipy import constants

from astrohack.dio import open_image

alma_image = resultsfolder+'/alma.split.image.zarr'

plt.close('all')

chan = 0

ds = open_image("results/alma.split.image.zarr")['ant_DV13']['ddi_0']

wavelength = scipy.constants.speed_of_light/ds.chan.values[chan]
l = ds.l.values
m = ds.m.values
u = ds.u.values*wavelength
v = ds.v.values*wavelength

print(u.min(),u.max())

plt.figure()
plt.imshow(np.abs(ds.BEAM[0,chan,0,:,:]),extent=[l.min(), l.max(), m.min(), m.max()])
plt.colorbar()
plt.show()

circle = patches.Circle((0,0), 12.5, fill=False, color='white', alpha=0.7, linewidth=2)

fig, ax =plt.subplots()
plt.imshow(np.abs(ds.APERTURE[0,chan,0,:,:]),extent=[u.min(), u.max(), v.min(), v.max()])

plt.show()

ds

In [ ]:
from astrohack.panel import panel

panel_model = 'rigid'

alma_panel = panel(
    image_name=alma_image, 
    panel_model=panel_model,
    parallel=True,
    overwrite=True
)

In [ ]:
import numpy as np
from astrohack.dio import open_panel

def relative_difference(mean, expected):  
    return 2*np.abs(mean - expected)/(abs(mean) + abs(expected))

def verify_panel_positions(
    panel_list=['3-11', '5-31', '7-52', '11-62'], 
    reference_position = np.array([-2.39678052, -0.87024129, 0.89391852, 0.48643069]),
    antenna='ant_DV13',
    ddi='ddi_0'
):
    
    M_TO_MILS = 39370.1
    
    panel_mds = open_panel('results/alma.split.panel.zarr')
    
    panel_position = np.mean(panel_mds[antenna][ddi].sel(labels=panel_list).PANEL_SCREWS.values*M_TO_MILS, axis=1)

    relative_position = relative_difference(panel_position, reference_position)
    
    if np.any(relative_position > 1e-6): 
        print("There were changes!")
        print(relative_position)    

In [ ]:
verify_panel_positions(antenna='ant_DV13', ddi='ddi_0')